# Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Wasington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour. 

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value. 

In [1]:
import pandas as pd
data = pd.read_csv('../data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Add some features from the past

Since we have time stamp of every measurement, we can see the data as a time series and use data from the past. We add one feature column computed from the data of the previous hour.

In [2]:
data.sort_values(['dteday', 'hr'])
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


## Neural Network regressor

Implement a neural network regressor based on all reasonable features from the input data set. Notice that some of the features from the input data cannot be used.

### Data preparation

Prepare train and test data sets.

In [3]:
from sklearn.model_selection import train_test_split

X_all = data[['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']


X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y_all,
    random_state=1,
    test_size=0.2)

print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

Train size: 13902
Test size: 3476


Standardize the features

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Training a regressor
Design and train a regression model. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/) and [optimizers](https://keras.io/optimizers/).

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()

model.add(Dense(32, input_shape=(12, )))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))

Using TensorFlow backend.


Compile the model

In [6]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [7]:
model.fit(X_train, y_train,
          batch_size = 128, epochs = 500, verbose=1,
          validation_data=(X_test, y_test))

Train on 13902 samples, validate on 3476 samples
Epoch 1/500
13902/13902 [==============================] - 0s 18us/step - loss: 67892.2894 - mean_absolute_error: 188.5114 - val_loss: 66969.1189 - val_mean_absolute_error: 187.8355
Epoch 2/500
13902/13902 [==============================] - 0s 12us/step - loss: 65537.5564 - mean_absolute_error: 184.7933 - val_loss: 64226.7073 - val_mean_absolute_error: 180.7883
Epoch 3/500
13902/13902 [==============================] - 0s 12us/step - loss: 62810.0468 - mean_absolute_error: 176.6086 - val_loss: 61948.0139 - val_mean_absolute_error: 174.1956
Epoch 4/500
13902/13902 [==============================] - 0s 12us/step - loss: 60909.1161 - mean_absolute_error: 172.1114 - val_loss: 60411.7278 - val_mean_absolute_error: 171.1968
Epoch 5/500
13902/13902 [==============================] - 0s 13us/step - loss: 59465.7962 - mean_absolute_error: 169.3364 - val_loss: 59069.8619 - val_mean_absolute_error: 168.7315
Epoch 6/500
13902/13902 [================

13902/13902 [==============================] - 0s 12us/step - loss: 29807.5986 - mean_absolute_error: 107.5347 - val_loss: 29054.4496 - val_mean_absolute_error: 105.1427
Epoch 46/500
13902/13902 [==============================] - 0s 12us/step - loss: 29360.7928 - mean_absolute_error: 106.3627 - val_loss: 28583.1710 - val_mean_absolute_error: 103.8674
Epoch 47/500
13902/13902 [==============================] - 0s 12us/step - loss: 28897.1304 - mean_absolute_error: 105.5458 - val_loss: 28143.9439 - val_mean_absolute_error: 103.2400
Epoch 48/500
13902/13902 [==============================] - 0s 11us/step - loss: 28431.9878 - mean_absolute_error: 104.6475 - val_loss: 27699.6252 - val_mean_absolute_error: 102.2754
Epoch 49/500
13902/13902 [==============================] - 0s 11us/step - loss: 28039.0494 - mean_absolute_error: 104.0323 - val_loss: 27249.8662 - val_mean_absolute_error: 101.1415
Epoch 50/500
13902/13902 [==============================] - 0s 11us/step - loss: 27731.2385 - mean

13902/13902 [==============================] - 0s 13us/step - loss: 17374.7140 - mean_absolute_error: 84.8364 - val_loss: 15549.0504 - val_mean_absolute_error: 77.0408
Epoch 91/500
13902/13902 [==============================] - 0s 11us/step - loss: 17241.6323 - mean_absolute_error: 84.4486 - val_loss: 15379.9912 - val_mean_absolute_error: 76.7181
Epoch 92/500
13902/13902 [==============================] - 0s 11us/step - loss: 16936.2110 - mean_absolute_error: 83.9633 - val_loss: 15216.0191 - val_mean_absolute_error: 76.6351
Epoch 93/500
13902/13902 [==============================] - 0s 11us/step - loss: 16780.3739 - mean_absolute_error: 83.5472 - val_loss: 15049.5093 - val_mean_absolute_error: 76.6274
Epoch 94/500
13902/13902 [==============================] - 0s 11us/step - loss: 16648.8357 - mean_absolute_error: 83.3773 - val_loss: 14912.7314 - val_mean_absolute_error: 76.3723
Epoch 95/500
13902/13902 [==============================] - 0s 11us/step - loss: 16412.7260 - mean_absolute_

13902/13902 [==============================] - 0s 12us/step - loss: 13450.7558 - mean_absolute_error: 78.1899 - val_loss: 11155.7120 - val_mean_absolute_error: 68.9806
Epoch 136/500
13902/13902 [==============================] - 0s 12us/step - loss: 13367.6966 - mean_absolute_error: 78.0409 - val_loss: 11119.2298 - val_mean_absolute_error: 69.5170
Epoch 137/500
13902/13902 [==============================] - 0s 11us/step - loss: 13324.1778 - mean_absolute_error: 78.1528 - val_loss: 11051.8459 - val_mean_absolute_error: 68.5088
Epoch 138/500
13902/13902 [==============================] - 0s 11us/step - loss: 13326.9997 - mean_absolute_error: 78.1355 - val_loss: 11015.4671 - val_mean_absolute_error: 68.8013
Epoch 139/500
13902/13902 [==============================] - 0s 11us/step - loss: 13262.9498 - mean_absolute_error: 77.7673 - val_loss: 10964.2611 - val_mean_absolute_error: 68.7024
Epoch 140/500
13902/13902 [==============================] - 0s 12us/step - loss: 13312.8358 - mean_abso

13902/13902 [==============================] - 0s 12us/step - loss: 11973.9702 - mean_absolute_error: 74.7979 - val_loss: 9871.7664 - val_mean_absolute_error: 66.1943
Epoch 181/500
13902/13902 [==============================] - 0s 11us/step - loss: 12233.2370 - mean_absolute_error: 75.6606 - val_loss: 9896.1746 - val_mean_absolute_error: 65.7294
Epoch 182/500
13902/13902 [==============================] - 0s 11us/step - loss: 12162.2245 - mean_absolute_error: 75.4411 - val_loss: 9788.8265 - val_mean_absolute_error: 66.3133
Epoch 183/500
13902/13902 [==============================] - 0s 12us/step - loss: 12114.0166 - mean_absolute_error: 75.3685 - val_loss: 9769.5084 - val_mean_absolute_error: 66.2824
Epoch 184/500
13902/13902 [==============================] - 0s 11us/step - loss: 11995.1264 - mean_absolute_error: 74.6514 - val_loss: 9803.9393 - val_mean_absolute_error: 66.2260
Epoch 185/500
13902/13902 [==============================] - 0s 11us/step - loss: 11741.0564 - mean_absolute_

...

13902/13902 [==============================] - 0s 14us/step - loss: 11245.0363 - mean_absolute_error: 72.7173 - val_loss: 9158.6985 - val_mean_absolute_error: 64.4644
Epoch 316/500
13902/13902 [==============================] - 0s 13us/step - loss: 11454.9666 - mean_absolute_error: 73.2401 - val_loss: 9081.7475 - val_mean_absolute_error: 64.7061
Epoch 317/500
13902/13902 [==============================] - 0s 15us/step - loss: 11639.6449 - mean_absolute_error: 74.6065 - val_loss: 9116.0790 - val_mean_absolute_error: 63.4139
Epoch 318/500
13902/13902 [==============================] - 0s 14us/step - loss: 11296.0814 - mean_absolute_error: 72.5909 - val_loss: 9074.5036 - val_mean_absolute_error: 63.8883
Epoch 319/500
13902/13902 [==============================] - 0s 14us/step - loss: 11620.4730 - mean_absolute_error: 73.7316 - val_loss: 9058.0837 - val_mean_absolute_error: 65.3288
Epoch 320/500
13902/13902 [==============================] - 0s 12us/step - loss: 11508.6290 - mean_absolute_

13902/13902 [==============================] - 0s 12us/step - loss: 11299.9621 - mean_absolute_error: 72.5939 - val_loss: 9089.3600 - val_mean_absolute_error: 63.7403
Epoch 361/500
13902/13902 [==============================] - 0s 11us/step - loss: 11423.8606 - mean_absolute_error: 73.3721 - val_loss: 9032.6133 - val_mean_absolute_error: 64.3528
Epoch 362/500
13902/13902 [==============================] - 0s 12us/step - loss: 11256.5960 - mean_absolute_error: 72.6196 - val_loss: 9061.4719 - val_mean_absolute_error: 64.7100
Epoch 363/500
13902/13902 [==============================] - 0s 13us/step - loss: 11135.9846 - mean_absolute_error: 72.1834 - val_loss: 9001.0503 - val_mean_absolute_error: 64.8982
Epoch 364/500
13902/13902 [==============================] - 0s 11us/step - loss: 11243.7635 - mean_absolute_error: 72.6515 - val_loss: 9008.7652 - val_mean_absolute_error: 64.6838
Epoch 365/500
13902/13902 [==============================] - 0s 14us/step - loss: 11244.9796 - mean_absolute_

13902/13902 [==============================] - 0s 12us/step - loss: 11266.6372 - mean_absolute_error: 72.5565 - val_loss: 8982.5848 - val_mean_absolute_error: 65.2499
Epoch 406/500
13902/13902 [==============================] - 0s 11us/step - loss: 11216.4922 - mean_absolute_error: 72.7639 - val_loss: 8971.4743 - val_mean_absolute_error: 64.2358
Epoch 407/500
13902/13902 [==============================] - 0s 11us/step - loss: 11218.1357 - mean_absolute_error: 72.6440 - val_loss: 9046.0813 - val_mean_absolute_error: 64.1513
Epoch 408/500
13902/13902 [==============================] - 0s 11us/step - loss: 11238.0493 - mean_absolute_error: 72.9672 - val_loss: 9049.6647 - val_mean_absolute_error: 64.0857
Epoch 409/500
13902/13902 [==============================] - 0s 11us/step - loss: 11411.4369 - mean_absolute_error: 72.8785 - val_loss: 8949.8612 - val_mean_absolute_error: 64.5284
Epoch 410/500
13902/13902 [==============================] - 0s 12us/step - loss: 11255.1828 - mean_absolute_

13902/13902 [==============================] - 0s 13us/step - loss: 11287.9160 - mean_absolute_error: 72.7275 - val_loss: 8964.6784 - val_mean_absolute_error: 64.5480
Epoch 451/500
13902/13902 [==============================] - 0s 14us/step - loss: 10988.9809 - mean_absolute_error: 71.9644 - val_loss: 8973.2281 - val_mean_absolute_error: 63.6872
Epoch 452/500
13902/13902 [==============================] - 0s 14us/step - loss: 11210.6828 - mean_absolute_error: 72.6447 - val_loss: 8918.8858 - val_mean_absolute_error: 64.5329
Epoch 453/500
13902/13902 [==============================] - 0s 11us/step - loss: 11261.0649 - mean_absolute_error: 72.3506 - val_loss: 8937.5773 - val_mean_absolute_error: 63.3669
Epoch 454/500
13902/13902 [==============================] - 0s 12us/step - loss: 11218.5678 - mean_absolute_error: 72.2343 - val_loss: 8938.1056 - val_mean_absolute_error: 63.3899
Epoch 455/500
13902/13902 [==============================] - 0s 11us/step - loss: 11030.3222 - mean_absolute_

13902/13902 [==============================] - 0s 12us/step - loss: 11034.7368 - mean_absolute_error: 72.3396 - val_loss: 8937.8074 - val_mean_absolute_error: 64.3490
Epoch 496/500
13902/13902 [==============================] - 0s 11us/step - loss: 10982.4412 - mean_absolute_error: 72.1899 - val_loss: 8909.0494 - val_mean_absolute_error: 63.8778
Epoch 497/500
13902/13902 [==============================] - 0s 11us/step - loss: 11084.9057 - mean_absolute_error: 72.5364 - val_loss: 8908.3679 - val_mean_absolute_error: 64.3308
Epoch 498/500
13902/13902 [==============================] - 0s 11us/step - loss: 11205.5762 - mean_absolute_error: 72.2597 - val_loss: 9007.6236 - val_mean_absolute_error: 65.3685
Epoch 499/500
13902/13902 [==============================] - 0s 11us/step - loss: 11160.9107 - mean_absolute_error: 72.6274 - val_loss: 8956.5226 - val_mean_absolute_error: 63.5141
Epoch 500/500
13902/13902 [==============================] - 0s 11us/step - loss: 11008.5561 - mean_absolute_

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 190.1343498273878, std: 181.50427424823098
Test Root mean squared error: 94.67
Test Mean absolute error: 64.60


In [9]:
y_pred = model.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 94.83
Train Mean absolute error: 64.21
